In [1]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [592]:
import os
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, VGG19, Xception, InceptionV3
from tensorflow.keras.regularizers import l2


In [593]:
train_data_dir = "../data/processed/train"
val_data_dir = "../data/processed/val"

In [594]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range = 0.2, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [595]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(400,400),
    batch_size=8,
    class_mode="categorical"
)

Found 28 images belonging to 3 classes.


In [596]:
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size = (400,400),
    batch_size = 8,
    class_mode = "categorical"
)

Found 9 images belonging to 3 classes.


In [597]:
vgg16 = VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(400,400,3),
    name="vgg16",
)


In [598]:
for layer in vgg16.layers:
    layer.trainable = False

In [599]:
classesNum = len(os.listdir(train_data_dir))
classesNum

3

In [600]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.3),  # Increased rotation range
    tf.keras.layers.RandomZoom(0.3),  # Increased zoom range
    tf.keras.layers.RandomHeight(0.3),  # Increased height augmentation
    tf.keras.layers.RandomWidth(0.3),  # Increased width augmentation
    tf.keras.layers.RandomBrightness(0.3),  # Adjusts brightness by a factor of ±30%
    tf.keras.layers.RandomContrast(0.2),  # Adjusts contrast by a factor of ±20%
    tf.keras.layers.RandomRotation(0.2),  # Add some rotation to the data
    tf.keras.layers.GaussianNoise(0.1),  # Adds Gaussian noise with standard deviation of 0.1
], name="data_augmentation")

In [601]:
model = Sequential()
model.add(tf.keras.Input(shape=(400, 400, 3)))
model.add(data_augmentation)
model.add(vgg16)
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dense(512, activation="relu", kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(classesNum, activation="softmax", kernel_regularizer=l2(0.01))
)
print(model.summary())


Model: "sequential_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ data_augmentation (Sequential)  │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, None, None,     │    14,714,688 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_224         │ (None, None, None,     │         2,048 │
│ (BatchNormalization)            │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_27     │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,980,931 (57.15 MB)

 Trainable params: 265,219 (1.01 MB)

 Non-trainable params: 14,715,712 (56.14 MB)

None


In [602]:
model.compile(
optimizer=RMSprop(0.001), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [603]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs=50,
    verbose=1
)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.3208 - loss: 6.4771 - val_accuracy: 0.5000 - val_loss: 5.6030
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 150ms/step - accuracy: 0.3750 - loss: 5.4485 - val_accuracy: 1.0000 - val_loss: 4.7272
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.5312 - loss: 5.1256
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5000 - loss: 5.2341 - val_accuracy: 0.5000 - val_loss: 5.0598
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - accuracy: 0.4250 - loss: 5.4350 - val_accuracy: 1.0000 - val_loss: 3.9934
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.3750 - loss: 5.0740
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.5312 - loss: 4.6557 - val_accuracy: 0.5000 - val_loss: 4.3083
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 0.2500 - loss: 5.2229 - val_accuracy: 1.0000 - val_loss: 3.7918
Epoch 9/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.4427 - loss: 4.3093
Epoch 10/

In [604]:
def preprocess_image(image_path, image_size=(400,400)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

In [605]:
image_path = "test_pic/nis3.jpg"

preprocessed_image = preprocess_image(image_path)

In [606]:
predictions = model.predict(preprocessed_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step


In [607]:
predicted_class_index = np.argmax(predictions)
confidence = np.max(predictions)

In [608]:
class_labels = list(train_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_index]

print(f"Predicted class: {predicted_class_label} with confidence: {confidence}")

Predicted class: Serhan. with confidence: 0.5229313969612122


In [609]:
model.save('../models/saved_model/model.keras')

In [610]:
predictions

array([[0.33141539, 0.14565316, 0.5229314 ]], dtype=float32)

In [611]:
class_labels

['Nisa.', 'Sengul.', 'Serhan.']